In [1]:
import pandas, warnings

warnings.filterwarnings("ignore")

In [2]:
rating = pandas.read_table("ratings.csv", sep=",")

rating.iloc[:5]

,UserID,MovieID,Rating,Timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [3]:
rating.tail()

,UserID,MovieID,Rating,Timestamp
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352
100835,610,170875,3.0,1493846415


In [4]:
rating.isnull().sum().sum()

0

In [5]:
rating_table = rating.pivot_table(index="UserID", columns="MovieID", values="Rating")

rating_table.shape

(610, 9724)

In [6]:
rating_table.isnull().sum().sum()

5830804

In [7]:
rating_table.iloc[:5, :8]

MovieID,1,2,3,4,5,6,7,8
UserID,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
rating_table.size

5931640

In [9]:
SparseRate = rating_table.isnull().sum().sum() / rating_table.size

SparseRate = round(SparseRate * 100, 3)

SparseRate

98.3

Our Rating Table Have High Sparsity Rate at 98.3 %

So, Our Users Only Rate Very Small Percentage of Available Movies

In [10]:
WatchFilm = rating.groupby("MovieID")["Rating"].agg(["count", "mean"])

WatchFilm.columns = ["TotalWatch", "Rating"]

WatchFilm.iloc[:5]

,TotalWatch,Rating
MovieID,,
1,215,3.920930
2,110,3.431818
3,52,3.259615
4,7,2.357143
5,49,3.071429


In [11]:
WatchFilm.sort_values("Rating", ascending=False).iloc[:5]

,TotalWatch,Rating
MovieID,,
88448,1,5.0
100556,1,5.0
143031,1,5.0
143511,1,5.0
143559,1,5.0


In [12]:
WatchFilm.sort_values("TotalWatch", ascending=False).iloc[:5]

,TotalWatch,Rating
MovieID,,
356,329,4.164134
318,317,4.429022
296,307,4.197068
593,279,4.161290
2571,278,4.192446


Delete Any Movie that Watch Less Than 10

In [13]:
minimum_film_interaction = 10

WatchFilm = WatchFilm[WatchFilm["TotalWatch"] > minimum_film_interaction]

WatchFilm["TotalWatch"].min()

11

In [14]:
len(WatchFilm)

2121

In [15]:
rating["MovieID"].nunique()

9724

In [16]:
rating = rating[rating["MovieID"].isin(WatchFilm.index)]

rating["MovieID"].nunique()

2121

In [17]:
rating_table = rating.pivot_table(index="UserID", columns="MovieID", values="Rating")

SparseRate = rating_table.isnull().sum().sum() / rating_table.size

SparseRate = round(SparseRate * 100, 3)

SparseRate

93.845

Delete Any User that Watch Less Than 50 Movie

In [18]:
minimum_user_interaction = 50

UserWatch = rating.groupby("UserID")[["Rating"]].count()

UserWatch.iloc[:5]

,Rating
UserID,
1,209
2,26
3,24
4,177
5,44


In [19]:
UserWatch = UserWatch[UserWatch["Rating"] > minimum_user_interaction]

UserWatch["Rating"].min()

51

In [20]:
len(UserWatch)

352

In [21]:
rating = rating[rating["UserID"].isin(UserWatch.index)]

rating["UserID"].nunique()

352

In [22]:
rating_table = rating.pivot_table(index="UserID", columns="MovieID", values="Rating")

SparseRate = rating_table.isnull().sum().sum() / rating_table.size

SparseRate = round(SparseRate * 100, 3)

SparseRate

90.396

In [23]:
rating_table.size

746592

In [24]:
rating_table = rating_table.fillna(0)

rating_table.iloc[:5, :8]

MovieID,1,2,3,5,6,7,9,10
UserID,,,,,,,,
1,4.0,0.0,4.0,0.0,4.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,4.0,5.0,5.0,4.0,4.0,0.0,3.0
7,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
